In [10]:
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

import json
import pkg_resources

import numpy as np
import matplotlib.pyplot as plt
import descarteslabs as dl
from descarteslabs import workflows as wf
from tqdm.notebook import tqdm
import pandas as pd
from shapely import geometry as sgeom

import will_utils as utils

In [11]:
product_id = 'sentinel-2:L1C'
start_datetime='2015-01-01'
end_datetime='2021-10-01'
year=2018

In [12]:
#import yolo country
tomato_aoi = json.load(open('tomatoes_ndvi.geojson'))

# #Get UTM EPSG code
shapely_aoi = sgeom.shape(tomato_aoi['features'][0]['geometry'])
lat, lon = shapely_aoi.centroid.y, shapely_aoi.centroid.x
utm_epsg = utils.wgs_to_epsg(lat, lon)

#Create workflows context
wf_ctx = wf.GeoContext(
                geometry=tomato_aoi['features'][0]['geometry'],
                resolution=10.0,
                crs=f'EPSG:{utm_epsg}',
                bounds_crs='EPSG:4326')

# #Convert to a tile 
wf_ctx_data = wf_ctx.compute(progress_bar=True)
reflat, reflon = wf_ctx_data['bounds'][1], wf_ctx_data['bounds'][0]
tilesize= max(wf_ctx_data['arr_shape'])
tile = dl.scenes.DLTile.from_latlon(reflat, reflon, wf_ctx_data['resolution'], tilesize, 0)



Job ID: 9a6b6d92413b246482a06c7c4afc0033c25853c091f1e4b5
[      ] | Steps: 0/0 | Stage: SUCCEEDED                                      

Here we start to define the workflow. We will use Sentinel-2 imagery which has a resolution of 10 meters. In this next block of code, comments have been added before each step explaining what is happening in more detail.

In [14]:
def get_cdl(dltile, year, labels):
    if 255 in labels:
        assert labels[-1] == 255
        labels = labels[-1]

    if isinstance(dltile, dict):
        dltile, tilekey = dltile, dltile["properties"]["key"]
    elif isinstance(dltile, str):
        dltile, tilekey = dl.raster.dltile(dltile), dltile
    else:
        raise ValueError

    present_year = dt.datetime.now().year
    if not (2008 <= year < present_year):
        raise ValueError("We do not have CONUS-wide CDL before 2008")

    if False:
        ids = dl.metadata.ids(
            products=["usda:cdl:v1"],
            dltile=dltile,
            start_time="%i-01-01" % year,
            end_time="%i-01-01" % (year + 1),
        )
    else:
        # Avoid call to dl.metadata
        ids = ["usda:cdl:v1:meta_%i_30m_cdls_v1" % year]

    if len(ids) == 0:
        raise ValueError("CDL does not cover dltile %s for year %i" % (tilekey, year))

    try:
        array, _ = dl.raster.ndarray(ids, dltile=dltile)
    except Exception as error:
        print("ids:", ids)
        print("dltile:", tilekey)
        raise error

    # Set non-requested labels > 0 to 255 (other)
    other = np.in1d(array, np.array(labels), invert=True).reshape(array.shape)
    valid = array > 0
    array[other & valid] = 255

    return array

In [16]:
get_cdl(tile, 2020, range(1,255))

ValueError: 